In [1]:
import random
import networkx  
import pgmpy.models as pgmm  
import pgmpy.factors.discrete as pgmf              
import pgmpy.inference as pgmi
import random
import numpy
import emoji

mina = emoji.emojize(':skull:')
casilla_sin_pulsar = emoji.emojize(':white_large_square:')

In [2]:
#Entrada de datos del buscaminas
filas = 5
columnas = 5
minas = 10

In [3]:
# Ahora, debemos de crear el tablero del juego, el cual dependerá del numero de filas, columnas y minas
def tablero_buscaminas(filas, columnas, minas):
    # Antes de nada, se debe de comprobar que el número de minas existente en el tablero sea menor o igual al número de casillas
    if(minas > filas * columnas):
        raise Exception ("El número de minas debe ser menor o igual a" + filas*columnas)
        
    # A continuación se pasa a construir el tablero
    tableroBuscaminas = [[0 for x in range(filas)] for y in range(columnas)]
    
    # Vamos a pintar las minas. Recorremos el número de minas que queremos pintar y en el tablero mostramos "X" que indica
    # que en esa casilla existe una mina.
    for i in range(minas):
        tableroBuscaminas[random.randint(0,filas-1)][random.randint(0,columnas-1)] = mina
        
    # Ahora pasamos a poner los índices de las casillas colindantes con minas. El número que aparece en la casilla va en un
    # rango de 1 a 8 (arriba, abajo, izquierda, derecha y las 4 diagonales).
    minas_colindantes = 0
    for fila in range(filas):
        for columna in range(columnas):
            if(fila > 0 and tableroBuscaminas[fila-1][columna] == mina): # La mina se encuentra arriba
                minas_colindantes += 1
            if(fila +1 < len(tableroBuscaminas) and tableroBuscaminas[fila+1][columna] == mina): # La mina se encuentra abajo
                minas_colindantes +=1
            if(columna+1 < len(tableroBuscaminas[0]) and tableroBuscaminas[fila][columna+1] == mina):
                # La mina se encuentra a la derecha
                minas_colindantes +=1
            if(columna > 0 and tableroBuscaminas[fila][columna-1] == mina): # La mina se encuentra a la izquierda
                minas_colindantes +=1
            if(fila > 0 and columna+1 < len(tableroBuscaminas[0]) and tableroBuscaminas[fila-1][columna+1] == mina):
                # La mina se encuentra en la diagonal superior derecha
                minas_colindantes +=1
            if(fila+1 < len(tableroBuscaminas) and columna+1 < len(tableroBuscaminas[0]) and 
                   tableroBuscaminas[fila+1][columna+1] == mina): 
                # La mina se encuentra en la diagonal inferior derecha
                minas_colindantes +=1
            if(fila > 0 and columna > 0 and tableroBuscaminas[fila-1][columna-1] == mina):
                # La mina se encuentra en la diagonal superior izquierda
                minas_colindantes +=1
            if(fila+1 < len(tableroBuscaminas) and columna > 0 and tableroBuscaminas[fila+1][columna-1] == mina): 
                # La mina se encuentra en la diagonal inferior izquierda
                minas_colindantes +=1
            # Pasamos a guardar el índice de minas colindantes en la casilla del tablero donde no hay ninguna mina.
            if(minas_colindantes > 0 and tableroBuscaminas[fila][columna] != mina ):
                tableroBuscaminas[fila][columna] = str(minas_colindantes)
            minas_colindantes=0
            
    return tableroBuscaminas
    
    
    
    

In [4]:
# Prueba de creación del tablero con los parámetros indicados
tableroBuscaminas = tablero_buscaminas(filas,columnas,minas)
# Para decirle que es un array y que salg
print(numpy.array(tableroBuscaminas))

[['💀' '2' '💀' '💀' '1']
 ['2' '4' '4' '4' '2']
 ['1' '💀' '💀' '3' '💀']
 ['2' '3' '3' '3' '💀']
 ['1' '💀' '1' '1' '1']]


In [5]:
# Pasamos a ocultar las casillas del tablero, se determina que dichas casillas ocultas se muestran con el símbolo "O"
def ocultar_casillas_tablero(filas,columnas):
    tablero_buscaminas_oculto = [[casilla_sin_pulsar for columna in range(columnas)] for fila in range(filas)]
        
    return tablero_buscaminas_oculto

In [6]:
# Prueba de ocultación del tablero
tablero_buscaminas_oculto = ocultar_casillas_tablero(filas,columnas)
print(numpy.array(tablero_buscaminas_oculto))

[['⬜' '⬜' '⬜' '⬜' '⬜']
 ['⬜' '⬜' '⬜' '⬜' '⬜']
 ['⬜' '⬜' '⬜' '⬜' '⬜']
 ['⬜' '⬜' '⬜' '⬜' '⬜']
 ['⬜' '⬜' '⬜' '⬜' '⬜']]


In [7]:
# Vamos ahora a destapar las casillas del tablero. Al clicar sobre una casilla que no contiene una mina, se van a destapar un
# conjunto colindante de casillas que no tienen minas ni están proximas con otras, es decir, con índice = 0. Como máximo,
# el límite del destapado va a ser si la casilla que se destapa tiene al menos un vecino con mina.
def destapar_casillas_tablero(fila,columna):
    # Guardo el valor de la casilla en la que hacemos click
    casillaDestapada = tableroBuscaminas[fila][columna]
    # Guardamos en una variable la longitud del tablero para optimizar código
    longitudTablero = len(tablero_buscaminas_oculto)
    
    # Si la casilla pulsada es una mina:
    if(casillaDestapada == mina):
        print(numpy.array(tableroBuscaminas))
        raise Exception("Has encontrado una mina. Has perdido")
    # Si no lo es, cambiamos su valor en el tablero oculto por el valor de la casilla destapada
    else:
        tablero_buscaminas_oculto[fila][columna] = casillaDestapada
        
    if(casillaDestapada == 0): #Comprobamos si la casilla que destapamos tiene 0 minas colindantes
        # si se da el caso, destapamos todas las casillas colindantes a la seleccionada ya que sabemos que ninguna es una mina
        # se comprueba que las colindantes estén dentro del tablero
        if((fila-1)>=0):
            destapar_casillas_tablero(fila-1,columna)
        if((fila+1)< longitudTablero):
            destapar_casillas_tablero(fila+1,columna)
        if((columna+1)< longitudTablero):
            destapar_casillas_tablero(fila,columna+1)
        if((columna-1)>=0):
            destapar_casillas_tablero(fila,columna-1)
        if((fila-1)>=0 and (columna+1) < longitudTablero):
            destapar_casillas_tablero(fila-1,columna+1)
        if((fila+1)< longitudTablero and (columna+1)< longitudTablero):
            destapar_casillas_tablero(fila+1,columna+1)
        if((fila-1)>=0 and (columna-1)>= 0):
            destapar_casillas_tablero(fila-1,columna-1)
        if((fila+1)< longitudTablero and (columna-1)>=0):
            destapar_casillas_tablero(fila+1,columna-1)
    
    return tablero_buscaminas_oculto        
            
    
            

In [8]:
# Vamos a probar ahora el destapado de una casilla
destapar_casillas_tablero(2,4)

[['💀' '2' '💀' '💀' '1']
 ['2' '4' '4' '4' '2']
 ['1' '💀' '💀' '3' '💀']
 ['2' '3' '3' '3' '💀']
 ['1' '💀' '1' '1' '1']]


Exception: Has encontrado una mina. Has perdido

In [14]:
# Creamos un método para que se muestre los índices de las casillas vecinas a la destapada anteriormente. 
# Recorre las casillas colindantes a la casillas destapada y devuelve un array con la posición donde busca los indices.


def posicion_casillas_vecinas(fila,columna,tableroBuscaminas):
    if(tableroBuscaminas[fila][columna] == mina):
        raise Exception("Has encontrado una mina. Has perdido")
    else:
        indicesCasillas = []
        if((fila-1)>=0 and columna>=0):
            indicesCasillas.append("{0}{1}".format(fila-1, columna))
        if((fila+1)< len(tableroBuscaminas) and columna>=0):
            indicesCasillas.append("{0}{1}".format(fila+1, columna))
        if((fila>=0 and (columna+1) < len(tableroBuscaminas))):
            indicesCasillas.append("{0}{1}".format(fila, columna+1))
        if(fila>=0 and (columna-1)>=0):
            indicesCasillas.append("{0}{1}".format(fila, columna-1))
        if(((fila-1)>=0 and (columna+1) < len(tableroBuscaminas))):
            indicesCasillas.append("{0}{1}".format(fila-1, columna+1))
        if((fila+1)< len(tableroBuscaminas) and (columna+1)< len(tableroBuscaminas)):
            indicesCasillas.append("{0}{1}".format(fila+1, columna+1))
        if((fila-1)>=0 and (columna-1)>=0):
            indicesCasillas.append("{0}{1}".format(fila-1, columna-1))
        if((fila+1)< len(tableroBuscaminas) and (columna-1)>=0):
            indicesCasillas.append("{0}{1}".format(fila+1, columna-1))
    
    return indicesCasillas

In [26]:
# Ejemplo en relación con el anterior en el que se detapa la casilla. Puede servirnos en un futuro
posicion_casillas_vecinas(0,1,tableroBuscaminas)

['11', '02', '00', '12', '10']

In [27]:
def listado_de_minas(tableroBuscaminas):
    listado_minas = [] # Vamos a obtener las casillas donde están las minas situadas
    for fila in range(filas):
        for columna in range(columnas):
            if(tableroBuscaminas[fila][columna] == mina):
                listado_minas.append('{0},{1}'.format(fila,columna))
    return listado_minas


In [28]:
listado_de_minas(tableroBuscaminas)

['0,0', '0,2', '0,3', '2,1', '2,2', '2,4', '3,4', '4,1']

In [32]:
# Pasamos a crear la red bayesiana
def red_bayesiana(filas,columnas,tableroBuscaminas):
    Buscaminas_bayesiano = pgmm.BayesianModel()
    """En primer lugar, debemos de crear los nodos de la red bayesiana,
    los cuales hay tantos como variable aleatorias haya. En este caso,
    va a haber dos: X e Y
    X: presencia de una mina en una casilla. [True,False]
    Y: cantidad de minas que hay en la casilla destapada [0-8], depende
    del número de variables colindantes X"""
    
    x_totales = [] # casillas ocultas totales del tablero
    x_ocultas = [] # casillas ocultas con vecinos destapados
    y_destapadas = []
    casilla_oculta_sin_colidantes_destapados = False
    
    for fila in range(filas):
        for columna in range(columnas):
            if(tableroBuscaminas[fila][columna] == casilla_sin_pulsar):
                x = 'Tapada{0}{1}'.format(fila,columna) # lo pone en formato Tapada (0,1),Tapada (0,2)...
                x_totales.append(x) # Va metiendo las casillas sin pulsar (oculta) en la lista de casillas totales ocultas
                
                casilla_tiene_vecinos_destapados = False
                
                casillas_vecinas = posicion_casillas_vecinas(fila,columna,tableroBuscaminas) #obtiene la posicion de las casillas vecinas
                
                for vecino in casillas_vecinas: # Recorro todas las casillas vecinas de la casilla en la que estamos actualmente
                    if(tableroBuscaminas[int(vecino[0])][int(vecino[1])] != casilla_sin_pulsar): # comprueba que los vecinos de la casilla estan destapados
                        casilla_tiene_vecinos_destapados = True
                        break
                if(not casilla_oculta_sin_colindantes_destapados or casilla_tiene_vecinos_destapados): # Comprobamos que la casilla actual tiene vecino destapado
                    x_ocultas.append(x) # Añadimos la casilla a la lista de casillas tapadas con vecinos destapados
                    Buscaminas.bayesiano.add_nodes_from([x]) # Se añade la casilla como nodo de nuestra red
                
                if(not casilla_tiene_vecinos_destapados):
                    casilla_oculta_sin_colindates_destapados = True
            else:
                y = 'Destapada{0}{1}'.format(fila,columna) # lo pone en formato Destapada (0,1),Destapada (0,2)...
                casilla_tiene_vecinos_ocultos = False # Inicialización de variable
                casillas_vecinas = posicion_casillas_vecinas(fila,columna,tableroBuscaminas) #obtiene la posicion de las casillas vecinas
                Listado_de_minas = listado_de_minas(tableroBuscaminas) # Variable que contiene la posición de las minas del tablero
                for vecino in casillas_vecinas:
                    casilla_vecina_oculta = 'Tapada{0}{1}'.format(vecino[0],vecino[1])
                    if(tableroBuscaminas[int(vecino[0])][int(vecino[1])] == casilla_sin_pulsar and 
                       not Listado_de_minas.__contains__(casilla_vecina_oculta)):
                        casilla_tiene_vecinos_ocultos = True
                        break
                        
                if(casilla_tiene_vecinos_ocultos):
                    y_destapadas.append(y)
                    Buscaminas.bayesiano.add_nodes_from([y])
                    
    print("Lista del nodo X:")   
    print(x_ocultas)
    print("\n")
    print("Lista del nodo Y:")  
    print(y_destapadas)
    print("\n")
    print("Lista de minas conocidas:")
    print(Minas_conocidas)
        

Lista del nodo X:


NameError: name 'x_ocultas' is not defined